# Seminarios de Procesos Gaussianos

### Grupo de procesamiento de la información visual (VIP) 

<div style="text-align: right"> Miguel López Pérez </div>

# 2. Regresión

In [ ]:
import gpflow
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn import model_selection
import sklearn.datasets
import sklearn.metrics
import sklearn.ensemble
%matplotlib inline

In [ ]:
def plot(m):
    xx = np.linspace(-0.1, 1.1, 100).reshape(100, 1)
    mean, var = m.predict_y(xx)
    plt.figure(figsize=(12, 6))
    plt.plot(X, Y, 'kx', mew=2)
    plt.plot(xx, mean, 'C0', lw=2)
    plt.fill_between(xx[:,0],
                     mean[:,0] - 2*np.sqrt(var[:,0]),
                     mean[:,0] + 2*np.sqrt(var[:,0]),
                     color='C0', alpha=0.2)
    plt.xlim(-0.1, 1.1)

## Ejemplo

Nuestros datos observados van a venir de una suma de funciones trigonométricas con ruido en 12 puntos arbitrarios. 

In [ ]:
np.random.seed(20)
N = 12
X = np.random.rand(N,1)
Y = np.sin(12*X) + 0.66*np.cos(25*X) + np.random.randn(N,1)*0.1 + 3

plt.plot(X, Y, 'kx', mew=2)

Nuestro proceso gaussiano será capaz de calcular una distribución normal multivariante sobre nuestros puntos de entrenamiento. Además dado cualquier otro punto (de test) calculará la distribución normal multivariante sobre este punto de test y los de training. 

La varianza del likelihood sería el ruido que observa en los datos.

In [ ]:
sc = 0.1
var = 10
var_lik = 0.01
k = gpflow.kernels.RBF(1, lengthscales=sc, variance = var)
m = gpflow.models.GPR(X, Y, kern=k)
m.likelihood.variance = var_lik

plot(m)

**Pregunta**: ¿Cómo afectan los parámetros del kernel y de la verosimilitud al modelo?

### Optimización del modelo

Queremos encontrar los parámetros óptimos para ello los obtendremos por máxima verosimilitud. Optimizaremos los parámetros del modelo maximizando la evidencia:

$\hat{\theta} = \arg\max_\theta p(y\mid \theta)$ 

Podemos interpretar esto como que la búsqueda de los parámetros que hagan máxima la verosimilitud ("probabilidad") de haber observado nuestros datos.



In [ ]:
gpflow.train.ScipyOptimizer().minimize(m, maxiter=200)
plot(m)

In [ ]:
gpflow.training.AdamOptimizer(0.01).minimize(m, maxiter=200)
plot(m)

Parámetros estimados en nuestro modelo

In [ ]:
m.as_pandas_table() 

La incertidumbre que hemos obtenido (cuanto más grande mejor (es la log(verosimilitud))

In [ ]:
m.compute_log_likelihood()

¿Cuál sería la predicción en  $x = 0.5$? ¿y en $x=1$?

In [ ]:
mu, var = m.predict_y(np.array([0.5, 1]).reshape(2,1))

In [ ]:
print("El punto 0.5 tiene una distribución normal con media", mu[0][0], "y varianza ", var[0][0], 
      "\nEl punto x = 1 tiene una distribución normal con media", mu[1][0], "y varianza ", var[1][0])

In [ ]:
m.likelihood.variance = 0.1
m.likelihood.variance.trainable = False
gpflow.train.ScipyOptimizer().minimize(m, maxiter=200)

plot(m)

In [ ]:
m.read_trainables()

In [ ]:
m.compute_log_likelihood()

## Capacidad de generalización

Efectivamente, nuestra función predictiva se ajusta muy bien a nuestros datos observados de entrenamiento. Pero, ¿realmente está capturando la forma de la función latente? En otras palabras, ¿nuestro modelo generaliza?

En esta sección, vamos a pintar en rojo la función latente de modo que vemos como de bien nuestro modelo la está reconstruyendo a partir de los datos ruidosos observados.

In [ ]:
np.random.seed(200)
N = 12
X = np.random.rand(N,1)
Y = np.sin(12*X) + 0.66*np.cos(25*X) + np.random.randn(N,1)*0.1 + 3

N = 100
X_test = np.linspace(-0.1, 1.1, 100).reshape(100, 1)
Y_test = np.sin(12*X_test) + 0.66*np.cos(25*X_test) + 3

plt.plot(X, Y, 'kx', mew=2 , label='datos ruidosos observados')
plt.plot(X_test, Y_test, color = 'r', lw = 0.5, label='función latente')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.plot()

In [ ]:
m = gpflow.models.GPR(X, Y, kern=k)
m.likelihood.variance = 0.01
gpflow.train.ScipyOptimizer().minimize(m, maxiter=200)
plot(m)
plt.plot(X, Y, 'kx', mew=2 , label='datos ruidosos observados')
plt.plot(X_test, Y_test, color = 'r', lw = 0.5, label='función latente')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.plot()
print(m.compute_log_likelihood())

** Pregunta: ** ¿Qué ocurre cuando borras la línea *m.likelihood.variance = 0.01* y vuelves a ejecutarlo? ¿Por qué?

## Regresión lineal

In [ ]:
m = gpflow.models.GPR(X, Y, kern=gpflow.kernels.Linear(1))
m.likelihood.variance = 0.01
gpflow.train.ScipyOptimizer().minimize(m, maxiter=200)
plot(m)
plt.plot(X, Y, 'kx', mew=2 , label='datos ruidosos observados')
plt.plot(X_test, Y_test, color = 'r', lw = 0.5, label='función latente')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.plot()

**Pregunta:** ¿Qué obtendríamos con el kernel Matern12?

## Ejemplo con el data set de *Diabetes* de sklearn

In [ ]:
X, y = sklearn.datasets.load_diabetes(return_X_y=True)
print('El tamaño de este dataset es', X.shape)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.4, random_state = 100)
m = gpflow.models.GPR(X_train, y_train.reshape(-1,1), kern=gpflow.kernels.RBF(X_train.shape[1]) + gpflow.kernels.White(X_train.shape[1], 1e-5))
m.likelihood.variance = 0.1
gpflow.train.ScipyOptimizer().minimize(m, maxiter=500)
print('La log-verosimilitud del modelo es', m.compute_log_likelihood())

In [ ]:
pred_gp, var = m.predict_y(X_test)
mse_gp = sklearn.metrics.mean_squared_error(y_test, pred_gp)

In [ ]:
rf_model = sklearn.ensemble.RandomForestRegressor(n_estimators = 200, max_depth = 12, random_state = 12)
rf_model.fit(X_train, y_train)
pred_rf = rf_model.predict(X_test)
mse_rf = sklearn.metrics.mean_squared_error(y_test, pred_rf)

In [ ]:
print('El mse del random forest es ', mse_rf, 'mientras que en el modelo GP es ', mse_gp)